<a href="https://colab.research.google.com/github/immischein/ML-bandgap/blob/main/Model_building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


# Data Imports and Processing


In [ ]:
# Load the JSON data
with open("materials_data.json") as f:
    materials_data = json.load(f)

In [ ]:
# Build tabular dataset
records = []
for m in materials_data:
    struct = Structure.from_dict(m["structure"])
    record = {
        "material_id": m["material_id"],
        "band_gap": m["band_gap"],
        "formula": m["formula_pretty"],
        "density": m["density"],
        "volume": m["volume"],
        "a": struct.lattice.a,
        "b": struct.lattice.b,
        "c": struct.lattice.c,
        "alpha": struct.lattice.alpha,
        "beta": struct.lattice.beta,
        "gamma": struct.lattice.gamma,
        "n_atoms": m["nsites"]
    }
    records.append(record)

# df = pd.DataFrame(records)
# df.to_csv("bandgap_dataset.csv", index=False)

# print(f"Saved tabular dataset to bandgap_dataset.csv with shape {df.shape}")

In [8]:
df = pd.DataFrame(records)

NameError: name 'pd' is not defined

In [4]:
class Bandgaps_Regression(torch.nn.Module):
    def __init__(self):
        super(Bandgaps_Regression, self).__init__()
        self.linear = torch.nn.Linear(2,1)

    def forward(self,x):
        y = torch.sigmoid(self.linear(x))
        return y

In [5]:
model = Bandgaps_Regression()

In [6]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [7]:

model.train()

for epoch in range(200000):
    optimizer.zero_grad()

    # forward pass
    Y_pred = model(X_tensor)

    # compute loss
    loss = criterion(Y_pred, T_tensor)
    if epoch% 1000 == 0:
        print(loss.item())
    # backward pass
    loss.backward()
    optimizer.step()

NameError: name 'X_tensor' is not defined